# Analisis de Egresos Hospitalarios

In [68]:
import importlib

import pandas as pd
import numpy as np
import polars as pl

import matplotlib.pyplot as plt
import seaborn as sns

import lectura_archivos as fr
import calculo_metricas as aux

from functools import reduce

sns.set_style()
plt.rcParams["figure.figsize"] = (12, 6)

importlib.reload(fr)
importlib.reload(aux)

<module 'calculo_metricas' from 'c:\\Users\\ppizarro\\Desktop\\Javier Rojas\\Javier\\trabajo\\SSMO\\EPH\\modulo_gestion\\1.analisis_problemas_de_salud\\egresos-hospitalarios\\calculo_metricas.py'>

In [67]:
HOSPITAL_A_ANALIZAR = 112103  # Este es el Torax

## Lectura de archivos

- Se leen todos los archivos que estan en input/utf-8. Los archivos estan separados por ;
- Se dejan solamente los egresos de los diagnosticos que **SI** tenga el Tórax
- Se cambia la codificacion de INTERV_Q de 2 (No, 1 es SI) a 0.
- Se cambia la codificacion de CONDICION_EGRESO (orig 1 = Vivo, 2 = Fallecido) de 1 -> 0 y 2 -> 1
- Se cambia la codificacion de GLOSA_REGION_RESIDENCIA {
                        "Del Libertador B. O'Higgins": "del Libertador General Bernardo O'Higgins",
                        "De Aisén del Gral. C. Ibáñez del Campo": "Aysén del General Carlos Ibáñez del Campo",
                    }

In [3]:
df = fr.leer_archivos(HOSPITAL_A_ANALIZAR)


# Analisis de Egresos Hospitalarios

Se calculan las siguientes metricas:

- Número de Egresos (n_egresos)
- Días de Estada promedio (dias_estada_promedio)
- Cantidad de Intervenciones Quirúrgicas (n_int_q)
- Cantidad de Muertes (n_muertos)

Todas estas metricas estan calculadas a nivel de Año, Establecimiento de Salud y Diagnóstico. Por
ejemplo:

***"En el año 2019, el Hospital San José (código XXXXXXX) tuvo XX egresos, con XX dias estada_promedio,
con XX intervenciones_quirúrgicas y XX muertos para el diagnóstico A24.4."***

Este análisis se realizó entre 2001 y 2020 para todos los hospitales/clínicas de Chile.

In [4]:
AGRUPACION = [
    "ANO_EGRESO",
    "ESTABLECIMIENTO_SALUD",
    "GLOSA_ESTABLECIMIENTO_SALUD",
    "DIAG1",
]

metricas = aux.obtener_metricas_egresos(df, AGRUPACION).collect(streaming=True)

# Analisis de Estratos

In [47]:
variables_a_rankear = ["n_egresos"]
subgrupo_del_ranking = ["ANO_EGRESO", "DIAG1"]
unir_tablas_por = AGRUPACION


In [6]:
dict_estratos = aux.obtener_diccionario_estratos(df, HOSPITAL_A_ANALIZAR)


In [48]:
dict_resultados_estratos = aux.obtener_resumen_por_estratos(
    metricas, dict_estratos, variables_a_rankear, subgrupo_del_ranking
)

resultado = reduce(aux.left_join_consecutivo, dict_resultados_estratos.values())


In [ ]:
cie = fr.leer_cie()

In [43]:
resultados_torax = resultado.filter(pl.col("ESTABLECIMIENTO_SALUD") == HOSPITAL_A_ANALIZAR)


In [52]:
diags_mas_relevantes = resultados_torax.filter(pl.col("ranking_nacionales_n_egresos") == 1).sort(
    by=["ANO_EGRESO", "n_egresos", "n_int_q"], descending=[False, True, True]
)

ANO_EGRESO,ESTABLECIMIENTO_SALUD,GLOSA_ESTABLECIMIENTO_SALUD,DIAG1,n_egresos,dias_estada_promedio,n_int_q,n_muertos,ranking_nacionales_n_egresos,total_nacionales_n_egresos,%_nacionales_n_egresos,ranking_publicos_n_egresos,total_publicos_n_egresos,%_publicos_n_egresos,ranking_privados_n_egresos,total_privados_n_egresos,%_privados_n_egresos,ranking_grd_n_egresos,total_grd_n_egresos,%_grd_n_egresos,ranking_interno_n_egresos,total_interno_n_egresos,%_interno_n_egresos
i64,i64,str,str,u32,f64,i64,i64,u32,u32,f64,u32,u32,f64,u32,u32,f64,u32,u32,f64,u32,u32,f64
2001,112103,"""Instituto Naci…","""I259""",634,8.525237,174,23,1,4747,0.133558,1,2801,0.226348,1,2580,0.245736,1,2552,0.248433,156,4985,0.127182
2001,112103,"""Instituto Naci…","""Z136""",207,2.811594,28,2,1,938,0.220682,1,525,0.394286,1,620,0.333871,1,524,0.395038,2,4985,0.041525
2001,112103,"""Instituto Naci…","""D381""",197,8.781726,61,7,1,586,0.336177,1,470,0.419149,1,313,0.629393,1,435,0.452874,190,4985,0.039519
2001,112103,"""Instituto Naci…","""J90X""",187,11.053476,54,9,1,1396,0.133954,1,1182,0.158206,1,401,0.466334,1,995,0.18794,89,4985,0.037513
2001,112103,"""Instituto Naci…","""C349""",181,12.983425,37,22,1,2017,0.089737,1,1338,0.135277,1,860,0.210465,1,1059,0.170916,211,4985,0.036309
2001,112103,"""Instituto Naci…","""J459""",126,7.722222,13,1,1,3726,0.033816,1,2956,0.042625,1,896,0.140625,1,2098,0.060057,99,4985,0.025276
2001,112103,"""Instituto Naci…","""J47X""",117,10.74359,13,1,1,591,0.19797,1,501,0.233533,1,207,0.565217,1,456,0.256579,98,4985,0.02347
2001,112103,"""Instituto Naci…","""I350""",85,9.0,30,1,1,500,0.17,1,371,0.229111,1,214,0.397196,1,355,0.239437,148,4985,0.017051
2001,112103,"""Instituto Naci…","""R042""",75,14.026667,18,6,1,579,0.129534,1,481,0.155925,1,173,0.433526,1,375,0.2,28,4985,0.015045


In [65]:
cie


Versión,Código,Descripción,Categoría,Sección,Capítulo,CodigoEgresos
str,str,str,str,str,str,str
"""CIE-v2013""","""A00""","""Cólera""","""A00 CÓLERA""","""A00-A09 ENFER…","""Cap.01 CIERTA…","""A00X"""
"""CIE-v2013""","""A00.0""","""Cólera debido …","""A00 CÓLERA""","""A00-A09 ENFER…","""Cap.01 CIERTA…","""A000"""
"""CIE-v2013""","""A00.1""","""Cólera debido …","""A00 CÓLERA""","""A00-A09 ENFER…","""Cap.01 CIERTA…","""A001"""
"""CIE-v2013""","""A00.9""","""Cólera, no esp…","""A00 CÓLERA""","""A00-A09 ENFER…","""Cap.01 CIERTA…","""A009"""
"""CIE-v2013""","""A01""","""Fiebres tifoid…","""A01 FIEBRES TI…","""A00-A09 ENFER…","""Cap.01 CIERTA…","""A01X"""
"""CIE-v2013""","""A01.0""","""Fiebre tifoide…","""A01 FIEBRES TI…","""A00-A09 ENFER…","""Cap.01 CIERTA…","""A010"""
"""CIE-v2013""","""A01.1""","""Fiebre paratif…","""A01 FIEBRES TI…","""A00-A09 ENFER…","""Cap.01 CIERTA…","""A011"""
"""CIE-v2013""","""A01.2""","""Fiebre paratif…","""A01 FIEBRES TI…","""A00-A09 ENFER…","""Cap.01 CIERTA…","""A012"""
"""CIE-v2013""","""A01.3""","""Fiebre paratif…","""A01 FIEBRES TI…","""A00-A09 ENFER…","""Cap.01 CIERTA…","""A013"""


In [44]:
torax = resultados_torax.to_pandas()
torax.to_csv(
    "output/ranking_torax_pais/posiciones_torax_polars.csv",
    sep=";",
    decimal=".",
    encoding="utf-8",
    index=False,
)


# Analisis Sociodemográfico

In [8]:
desglose_sociodemografico = [
    "ANO_EGRESO",
    "REGION_RESIDENCIA",
    "GLOSA_REGION_RESIDENCIA",
    "region_pais",
    "COMUNA_RESIDENCIA",
    "GLOSA_COMUNA_RESIDENCIA",
    "comuna_region_pais",
    "DIAG1",
]

variables_a_analizar = [
    "DIAG1",
    "SEXO",
    "EDAD_CATEGORIA",
    "PUEBLO_ORIGINARIO",
    "GLOSA_PAIS_ORIGEN",
    "PREVISION",
    "BENEFICIARIO",
]


In [9]:
MAPPING_SOCIODEMOGRAFICO = {
    "GLOSA_REGION_RESIDENCIA": {
        "Del Libertador B. O'Higgins": "del Libertador General Bernardo O'Higgins",
        "De Aisén del Gral. C. Ibáñez del Campo": "Aysén del General Carlos Ibáñez del Campo",
    },
    "SEXO": {1: "Hombre", 2: "Mujer", 3: "Intersex", 99: "Desconocido"},
    "PUEBLO_ORIGINARIO": {
        1: "MAPUCHE",
        2: "AYMARA",
        3: "RAPA NUI (PASCUENSE)",
        4: "LICAN ANTAI",
        5: "QUECHUA",
        6: "COLLA",
        7: "DIAGUITA",
        8: "KAWÉSQAR",
        9: "YAGÁN (YÁMANA)",
        10: "OTRO (ESPECIFICAR)",
        96: "NINGUNO",
    },
    "PREVISION": {
        1: "FONASA",
        2: "ISAPRE",
        3: "CAPREDENA",
        4: "DIPRECA",
        5: "SISA",
        96: "NINGUNA",
        99: "DESCONOCIDO",
    },
}


def mappear_columnas(df, dict_mapeo):
    tmp = df.clone()

    for variable, dict_cambio in dict_mapeo.items():
        tmp = tmp.with_columns(pl.col(variable).map_dict(dict_cambio, default=pl.col(variable)))

    return tmp


def agregar_columnas_localizacion(df):
    tmp = df.with_columns(
        (("Región " + pl.col("GLOSA_REGION_RESIDENCIA") + ", Chile").alias("region_pais"))
    )

    tmp = tmp.with_columns(
        (pl.col("GLOSA_COMUNA_RESIDENCIA") + ", " + pl.col("region_pais")).alias(
            "comuna_region_pais"
        )
    )

    return tmp


def agregar_categorizacion_edad(df):
    tmp = df.with_columns(
        (df.get_column("EDAD_A_OS"))
        .cut(bins=range(0, 101, 10), maintain_order=True)
        .select(pl.col("category").alias("EDAD_CATEGORIA"))
    )

    return tmp

In [10]:
with pl.StringCache():
    df_socio = pl.scan_csv("input/utf-8/*.csv", separator=";", dtypes=fr.DICT_VARIABLES).filter(
        pl.col("ESTABLECIMIENTO_SALUD") == HOSPITAL_A_ANALIZAR
    )

    df_socio = mappear_columnas(df_socio, MAPPING_SOCIODEMOGRAFICO)
    df_socio = agregar_columnas_localizacion(df_socio).collect()
    df_socio = agregar_categorizacion_edad(df_socio)

In [11]:
dict_resultados = {}

for variable in variables_a_analizar:
    agrupacion_total = desglose_sociodemografico.copy()
    if variable != "DIAG1":
        agrupacion_total += [variable]

    resultado = (
        df_socio.groupby(agrupacion_total)
        .agg(pl.col(variable).count().alias("conteo"))
        .sort(["ANO_EGRESO", "DIAG1", "conteo"], descending=True)
        .with_columns(
            pl.concat_str(pl.col(desglose_sociodemografico), separator="-").alias("llave_id")
        )
    )

    dict_resultados[variable] = resultado

In [12]:
import xlsxwriter

with xlsxwriter.Workbook("output/distribucion_sociodemografica/distribucion_polars.xlsx") as f:
    for var, df in dict_resultados.items():
        df.write_excel(workbook=f, worksheet=var)